In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 49.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4e8a5021c95022eff2d369091fa39713f39818fcc4bd4a8e96465bf6fa356e46
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
import time

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [ ]:
print(df.count(), len(df.columns))

3524477 29


In [ ]:
df = df.filter((df.modelMag_u != '-9999') & (df.modelMag_g != '-9999') & (df.modelMag_r != '-9999') & (df.modelMag_i != '-9999') & 
               (df.modelMag_z != '-9999') & (df.fiberMag_u != '-9999') & (df.fiberMag_g != '-9999') & (df.fiberMag_r != '-9999') & 
               (df.fiberMag_i != '-9999') & (df.fiberMag_z != '-9999') & (df.petroR50_r != '-9999') & (df.petroR90_r != '-9999') & 
               (df.petroR50_z != '-9999') & (df.petroR90_z != '-9999') & (df.r != '-9999') & (df.i != '-9999') & (df.z != '-9999') & 
               (df.redshift != '-9999') & (df.zerr != '-9999') & (df.mmug != '-9999') & (df.mmgr != '-9999') & (df.mmri != '-9999') & 
               (df.mmiz != '-9999') & (df.mfug != '-9999') & (df.mfgr != '-9999') & (df.mfri != '-9999') & (df.mfiz != '-9999'))
df.show(5)

+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|_c0|               objid|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z| redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+------

In [ ]:
print(df.count(), len(df.columns))

3524477 29


In [ ]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [ ]:
df = df.drop('objid', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  21.6

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+---------------

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)

In [ ]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[19.74829,19.4581...|       2|
|[23.74654,23.1965...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml import Pipeline
nb = NaiveBayes(labelCol="redshift", featuresCol="features")

In [ ]:
start_time = time.time()
model = nb.fit(train)
print("%s seconds" % (time.time() - start_time))

83.71453046798706 seconds


In [ ]:
pred_test = model.transform(test)

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test))

Naive Bayes Accu: 0.7995456744859011


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test))

test data (weightedPrecision):  0.8242098924108137
test data (weightedRecall):  0.7995456744859011


# Cascading

In [ ]:
final_pred = model.transform(final_df)
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.63269,21.2791...|       1|[-1387.1350793054...|[0.00776090157627...|       1.0|
|[19.74829,19.4581...|       2|[-1270.6028212066...|[0.09519426133401...|       1.0|
|[23.74654,23.1965...|       0|[-1444.5532189323...|[0.19594392644807...|       1.0|
|[20.63075,20.2542...|       1|[-1325.0398747347...|[0.10512706117194...|       1.0|
|[21.38126,21.1348...|       1|[-1392.8030810952...|[0.16389959397519...|       1.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
final_pred = final_pred.withColumn("objid", monotonically_increasing_id())
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.63269,21.2791...|       1|[-1387.1350793054...|[0.00776090157627...|       1.0|    0|
|[19.74829,19.4581...|       2|[-1270.6028212066...|[0.09519426133401...|       1.0|    1|
|[23.74654,23.1965...|       0|[-1444.5532189323...|[0.19594392644807...|       1.0|    2|
|[20.63075,20.2542...|       1|[-1325.0398747347...|[0.10512706117194...|       1.0|    3|
|[21.38126,21.1348...|       1|[-1392.8030810952...|[0.16389959397519...|       1.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
print(df.count(), len(df.columns))
new_df = df.join(final_pred, df.objid == final_pred.objid).select(df["*"],final_pred["prediction"])
print(df.count(), len(df.columns))
new_df.show(5)

3524477 28
3524477 28
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+-----------------+---------+------------------+------------------+---------+---------+---------+---------+---------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|        petroR90_r|petroR50_z|        petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|              mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+-----------------+---------+------------------+------------------

In [ ]:
#from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)

In [ ]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[20.90748,21.2373...|       1|
|[23.24831,21.2138...|       3|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
nb = NaiveBayes(labelCol="redshift", featuresCol="features2")

In [ ]:
model = nb.fit(train)

In [ ]:
pred_test2 = model.transform(test)

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test2))

Naive Bayes Accu: 0.7996573317477121


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test2))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test2))

test data (weightedPrecision):  0.823440150506004
test data (weightedRecall):  0.7996573317477121
